In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import ipyparallel
from ipyparallel import Client

In [3]:
import time

In [5]:
import ipy_executor

In [6]:
c = Client()
c.ids

[0, 1, 2, 3, 4, 5, 6, 7]

In [7]:
def f(x):
    import time
    time.sleep(0.1)
    return x**2

In [8]:
with ipy_executor.IpyExecutor(c) as ex:
    list(ex.map(f,range(30)))

In [9]:
def g(x):
    import time
    time.sleep(0.1)
    if x==17:
        raise ValueError("Oops!")
    return x**2

In [10]:
with ipy_executor.IpyExecutor(c) as ex:
    list(ex.map(g,range(50)))

RemoteError: ValueError(Oops!)